#GRU

#steup

In [1]:
import os
import cv2
import numpy as np
import string
import time
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
#from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.layers import Dense, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional

from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.compat.v1.keras.layers import CuDNNGRU
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, sigmoid, softmax

from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm import tqdm
from collections import Counter

from PIL import Image

In [2]:
import pickle
final_paths = pickle.load( open( "./data/image_path.p", "rb" ) )
final_texts = pickle.load( open( "./data/image_text.p", "rb" ) )

In [3]:
final_paths[:10], final_texts[:10]

(['words/mjsynth_sample/mjsynth_sample/36_Espoused_26563.jpg',
  'words/mjsynth_sample/mjsynth_sample/34_cloudbursts_14341.jpg',
  'words/mjsynth_sample/mjsynth_sample/35_Lubrication_45495.jpg',
  'words/mjsynth_sample/mjsynth_sample/33_PAVAROTTI_56019.jpg',
  'words/mjsynth_sample/mjsynth_sample/32_ASPIRANTS_4311.jpg',
  'words/mjsynth_sample/mjsynth_sample/30_electroplate_25005.jpg',
  'words/mjsynth_sample/mjsynth_sample/31_slanderer_71483.jpg',
  'words/mjsynth_sample/mjsynth_sample/29_oceanfronts_53003.jpg',
  'words/mjsynth_sample/mjsynth_sample/27_Pus_61444.jpg',
  'words/mjsynth_sample/mjsynth_sample/26_Pupa_61305.jpg'],
 ['Espoused',
  'cloudbursts',
  'Lubrication',
  'PAVAROTTI',
  'ASPIRANTS',
  'electroplate',
  'slanderer',
  'oceanfronts',
  'Pus',
  'Pupa'])

In [4]:
final_paths=final_paths[:50000]
final_texts=final_texts[:50000]

In [5]:
#del final_paths[25926]
#del final_texts[25926]

In [6]:
### get vocab from dataset
vocab = set("".join(map(str, final_texts)))
print(sorted(vocab))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [7]:
Counter("".join(map(str, final_texts)))


Counter({'0': 37,
         '1': 52,
         '2': 26,
         '3': 21,
         '4': 25,
         '5': 13,
         '6': 16,
         '7': 12,
         '8': 15,
         '9': 10,
         'A': 11556,
         'B': 3684,
         'C': 6814,
         'D': 6050,
         'E': 15856,
         'F': 2641,
         'G': 4542,
         'H': 3733,
         'I': 12009,
         'J': 494,
         'K': 1521,
         'L': 7589,
         'M': 4488,
         'N': 9654,
         'O': 8338,
         'P': 4899,
         'Q': 329,
         'R': 10523,
         'S': 13333,
         'T': 9501,
         'U': 4731,
         'V': 1694,
         'W': 1717,
         'X': 406,
         'Y': 2159,
         'Z': 663,
         'a': 20506,
         'b': 4495,
         'c': 9476,
         'd': 9245,
         'e': 30352,
         'f': 3104,
         'g': 7947,
         'h': 5767,
         'i': 23038,
         'j': 432,
         'k': 2634,
         'l': 13977,
         'm': 6588,
         'n': 19178,
         'o': 1

In [8]:
char_list = sorted(vocab)

def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)
        
    return dig_lst

In [9]:
encode_to_labels("HELLO")


[17, 14, 21, 21, 24]

In [10]:
import pickle
val_image_paths  = pickle.load( open( "./data/val_image_path.p", "rb" ) )
val_image_texts = pickle.load( open( "./data/val_image_text.p", "rb" ) )

In [11]:
val_final_paths = val_image_paths[int(len(val_image_paths) * 0.90) : ]
val_final_texts = val_image_texts[int(len(val_image_texts) * 0.90) : ]

In [12]:
train_final_paths = final_paths[ : int(len(final_paths) * 0.90)]
train_final_texts = final_texts[ : int(len(final_texts) * 0.90)]
#val_final_paths = final_paths[ : int(len(final_paths) * 0.90)]
#val_final_texts= final_texts[ : int(len(final_texts) * 0.90)]


In [13]:
train_final_paths

['words/mjsynth_sample/mjsynth_sample/36_Espoused_26563.jpg',
 'words/mjsynth_sample/mjsynth_sample/34_cloudbursts_14341.jpg',
 'words/mjsynth_sample/mjsynth_sample/35_Lubrication_45495.jpg',
 'words/mjsynth_sample/mjsynth_sample/33_PAVAROTTI_56019.jpg',
 'words/mjsynth_sample/mjsynth_sample/32_ASPIRANTS_4311.jpg',
 'words/mjsynth_sample/mjsynth_sample/30_electroplate_25005.jpg',
 'words/mjsynth_sample/mjsynth_sample/31_slanderer_71483.jpg',
 'words/mjsynth_sample/mjsynth_sample/29_oceanfronts_53003.jpg',
 'words/mjsynth_sample/mjsynth_sample/27_Pus_61444.jpg',
 'words/mjsynth_sample/mjsynth_sample/26_Pupa_61305.jpg',
 'words/mjsynth_sample/mjsynth_sample/25_Circularizing_13726.jpg',
 'words/mjsynth_sample/mjsynth_sample/24_Humps_37345.jpg',
 'words/mjsynth_sample/mjsynth_sample/23_creatives_17941.jpg',
 'words/mjsynth_sample/mjsynth_sample/22_Schoolchildren_68345.jpg',
 'words/mjsynth_sample/mjsynth_sample/21_reshaped_65134.jpg',
 'words/mjsynth_sample/mjsynth_sample/20_answerphones_3

In [14]:
len(train_final_paths), len(val_final_paths)


(45000, 54)

In [15]:
max_label_len = max([len(str(text)) for text in final_texts])

In [16]:
class My_Generator(Sequence):
    
    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        
        batch_paths = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_texts = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        images = []
        training_txt = []
        train_label_length = []
        train_input_length = []

        for im_path, text in zip(batch_paths, batch_texts):
            
            try:
                text = str(text).strip()
                img = cv2.cvtColor(cv2.imread(im_path), cv2.COLOR_BGR2GRAY)   

                ### actually returns h, w
                h, w = img.shape

                ### if height less than 32
                if h < 32:
                    add_zeros = np.ones((32-h, w)) * 255
                    img = np.concatenate((img, add_zeros))
                    h = 32

                ## if width less than 128
                if w < 128:
                    add_zeros = np.ones((h, 128-w)) * 255
                    img = np.concatenate((img, add_zeros), axis=1)
                    w = 128

                ### if width is greater than 128 or height greater than 32
                if w > 128 or h > 32:
                    img = cv2.resize(img, (128, 32))

                img = np.expand_dims(img , axis = 2)

                # Normalize each image
                img = img / 255.

                images.append(img)
                training_txt.append(encode_to_labels(text))
                train_label_length.append(len(text))
                train_input_length.append(31)
            except:
                
                pass

        return [np.array(images), 
               pad_sequences(training_txt, maxlen=max_label_len, padding='post', value=len(char_list)), 
               np.array(train_input_length), 
               np.array(train_label_length)], np.zeros(len(images))

In [17]:
train_final_paths[:1], train_final_texts[:1]


(['words/mjsynth_sample/mjsynth_sample/36_Espoused_26563.jpg'], ['Espoused'])

In [18]:
!nvidia-smi

Wed May  4 22:40:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
batch_size = 100
train_generator = My_Generator(train_final_paths, train_final_texts, batch_size)
val_generator = My_Generator(val_final_paths, val_final_texts, batch_size)

In [20]:
# input with shape of height=32 and width=128 
inputs = Input(shape=(32, 128, 1), name="image")


conv_1 = Conv2D(32, (3,3), activation = "selu", padding='same')(inputs)
pool_1 = MaxPool2D(pool_size=(2, 2))(conv_1)

conv_2 = Conv2D(64, (3,3), activation = "selu", padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2))(conv_2)

conv_3 = Conv2D(128, (3,3), activation = "selu", padding='same')(pool_2)
conv_4 = Conv2D(128, (3,3), activation = "selu", padding='same')(conv_3)

pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)

conv_5 = Conv2D(256, (3,3), activation = "selu", padding='same')(pool_4)

# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)

conv_6 = Conv2D(256, (3,3), activation = "selu", padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

conv_7 = Conv2D(64, (2,2), activation = "selu")(pool_6)

squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)

# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(CuDNNGRU(128, return_sequences=True))(squeezed)
blstm_2 = Bidirectional(CuDNNGRU(128, return_sequences=True))(blstm_1)

softmax_output = Dense(len(char_list) + 1, activation = 'softmax', name="dense")(blstm_2)

outputs = Dense(len(char_list) + 1, activation = 'softmax')(blstm_2)

act_model = Model(inputs, outputs)


In [21]:
act_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 32, 128, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 32, 128, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 32, 64)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 32, 128)        73856 

In [22]:
act_model.load_weights('./models/C_gru_best.hdf5')

In [23]:
labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
loss_out = Lambda(ctc_lambda_func, 
                  output_shape=(1,), 
                  name='ctc')([outputs, labels, input_length, label_length])

#model to be used at training time
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

In [24]:
def pre_process_image(path):
    
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)
    ### actually returns h, w
    h, w = img.shape

    ### if height less than 32
    if h < 32:
        add_zeros = np.ones((32-h, w)) * 255
        img = np.concatenate((img, add_zeros))
        h = 32

    ## if width less than 128
    if w < 128:
        add_zeros = np.ones((h, 128-w)) * 255
        img = np.concatenate((img, add_zeros), axis=1)
        w = 128

    ### if width is greater than 128 or height greater than 32
    if w > 128 or h > 32:
        img = cv2.resize(img, (128, 32))

    
    img = np.expand_dims(img , axis = 2)

    # Normalize each image
    img = img / 255.
    
    return img

In [25]:
def predict_output(img):
    # predict outputs on validation images
    prediction = act_model.predict(np.array([img]))
    ## shape (batch_size, num_timesteps, vocab_size)

    # use CTC decoder
    out = K.get_value(K.ctc_decode(prediction, 
                                   input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                                   greedy=True)[0][0])

    ## get the final text
    
    gg=''
    for x in out:

        print("predicted text = ", end = '')

        for p in x:
            
            if int(p) != -1 and gg=='':
                print(char_list[int(p)], end = '')
                gg=char_list[int(p)]
            elif int(p) != -1 :
               print(char_list[int(p)], end = '')
               gg=gg+char_list[int(p)]

        
        
        print('\n')

    #print(gg,"ggasdasd")  
    return gg

In [26]:
from IPython.display import display
from random import randrange

for i in range(1):
    if i ==1 :
      #index = randrange(len(val_final_paths))
      im = Image.open('./Reco_image/210_Flavor_29453.jpg')
      display(im)

      print("\n")

      test_img = pre_process_image('./Reco_image/210_Flavor_29453.jpg')
      predict_output(test_img)
    

In [27]:

im = Image.open('./Reco_image/210_Flavor_29453.jpg')
display(im)

print("\n")

test_img = pre_process_image('./Reco_image/210_Flavor_29453.jpg')
predict_output(test_img)



predicted text = Flavot



'Flavot'

In [28]:
holder =0
tt=[]
for i in os.listdir('./Reco_image'):
    
    #index = randrange(len(val_final_paths))
    im = Image.open('./Reco_image/'+str(i))
    display(im)

    print("\n")

    test_img = pre_process_image('./Reco_image/'+str(i))
    value= str(predict_output(test_img))
    tt.append(value)
  

print(holder)
  



predicted text = EXTRACT





predicted text = Flavot





predicted text = restricted





predicted text = Abnormal





predicted text = HEALER





predicted text = intercom





predicted text = UNTASTED

0


In [29]:
!pip install autocorrect

In [30]:
from autocorrect import Speller
spell = Speller(lang='en')
o=0
for i in range(len(tt)):
  #print(spell(tt[i].lower()),spell(val_final_texts[i].lower()))
  for k in range(len(val_final_texts)):
    if spell(tt[i].lower())==spell(val_final_texts[k].lower()):
      o+=1
print(o)

7


right now we are predicting all the words correctly but when we test it on testing data only 16 words were predcite correctly out of 50

#LSTM

In [31]:
# input with shape of height=32 and width=128 
inputs = Input(shape=(32, 128, 1))

conv_1 = Conv2D(16, (3,3), activation = 'relu', padding='same')(inputs)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(32, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

conv_3 = Conv2D(64, (3,3), activation = 'relu', padding='same')(pool_2)
conv_4 = Conv2D(64, (3,3), activation = 'relu', padding='same')(conv_3)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(64, (3,3), activation = 'relu', padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
 
conv_6 = Conv2D(64, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
 
conv_7 = Conv2D(64, (2,2), activation = 'relu')(pool_6)
 
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
 
# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(squeezed)
blstm_2 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(blstm_1)

outputs = Dense(len(char_list) + 1, activation = 'softmax')(blstm_2)

# model to be used at test time
act_model = Model(inputs, outputs)

In [32]:
os.listdir('./Reco_image')

['239_EXTRACT_27643.jpg',
 '210_Flavor_29453.jpg',
 '78_restricted_65363.jpg',
 '374_Abnormal_314.jpg',
 '85_HEALER_35423.jpg',
 '391_intercom_40141.jpg',
 '58_UNTASTED_83330.jpg']

In [33]:
act_model.load_weights('./models/C_LSTM_best.hdf5')

In [34]:
holder =0
tt=[]
for i in os.listdir('./Reco_image'):
    
    #index = randrange(len(val_final_paths))
    im = Image.open('./Reco_image/'+str(i))
    display(im)

    print("\n")

    test_img = pre_process_image('./Reco_image/'+str(i))
    value= str(predict_output(test_img))
    tt.append(value)
  

print(holder)



predicted text = EXTRACT





predicted text = Flavor





predicted text = restricted





predicted text = Abnormal





predicted text = HEALER





predicted text = intercon





predicted text = UNTASTED

0


In [35]:
from autocorrect import Speller
spell = Speller(lang='en')
o=0
for i in range(len(tt)):
  #print(spell(tt[i].lower()),spell(val_final_texts[i].lower()))
  for k in range(len(val_final_texts)):
    if spell(tt[i].lower())==spell(val_final_texts[k].lower()):
      o+=1
print(o)

7


same goes here .only 21 words were predcite correctly out of 50 in the testing dataset